In [105]:
import _pickle as cPickle
def unpickle(filename):
    f = open(filename, "rb")
    d = cPickle.load(f)
    f.close()
    return d


In [ ]:
#we import the final data to use ANN and KNN search to check the quality of the results

In [106]:
data=unpickle("data_new.pkl")

In [107]:
df=data[["title","final embeddings"]]

In [108]:
df=df.reset_index()

In [109]:
df.head(10)

,index,title,final embeddings
0,0,"Leviton 5050 B01-0-000 Electrical Receptacle, ...","[0.04236961156129837, 0.04030364751815796, -0...."
1,1,"Leviton 5206 50 Amp, 125/250 Volt, NEMA 10-50R...","[0.03194904699921608, 0.01351519301533699, -0...."
2,2,"Dundas Jafine CHK100ZW CHK100ZW6 Vents, 4-Inch...","[-0.04139666259288788, 0.1698998212814331, 0.5..."
3,3,"Holmes ""C"" Humidifier Filter, HWF65PDQ-U","[-0.03386007621884346, 0.09112260490655899, 0...."
4,4,"Holmes ""A"" Humidifier Filter, HWF62","[0.021733354777097702, 0.024240538477897644, -..."
5,5,"Holmes ""B"" Humidifier Filter, HWF64","[-0.03953782096505165, 0.1161935105919838, -0...."
6,6,Range Kleen 8234 Gas Range Knobs – 4 White Gas...,"[0.02836083248257637, 0.13297735154628754, 0.3..."
7,8,RANGE KLEEN RGP-200 Chrome Range Round Pan/Ora...,"[0.029942501336336136, -0.004961610771715641, ..."
8,9,RANGE KLEEN 106-A Chrome Range Pan/Green Label...,"[0.0237448588013649, 0.011799604631960392, -0...."
9,10,RANGE KLEEN R8-GE Chrome Range Trim Ring/Green...,"[-0.0164551492780447, 0.06439027190208435, 0.1..."


In [110]:
import numpy as np

In [111]:
for i in range(len(df["final embeddings"])):
    df["final embeddings"][i]=np.array(df["final embeddings"][i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [112]:
search_id=0

In [113]:
df["title"][search_id]

'Leviton 5050 B01-0-000 Electrical Receptacle, 125/250 Vac, 50 A, 3 Pole, 3 Wire, Pack of 1, Black'

In [114]:
df["final embeddings"][search_id].shape

(256,)

In [115]:
df.shape[0]

6040

In [116]:
import pandas as pd

In [ ]:
#KNN search performed

In [117]:
def simple_search(search_id,df):
    s_score=[]
    for i in range(df.shape[0]):
        a=np.linalg.norm(df["final embeddings"][search_id])
        b=np.linalg.norm(df["final embeddings"][i])
        c=a*b
        s_score.append(np.dot(df["final embeddings"][search_id],df["final embeddings"][i])/c)
    dataf=pd.DataFrame()
    dataf["similarity scores"]=s_score
    dataf["tag"]=df["title"]
    print(df["title"][search_id])
    print()
    print(dataf.nlargest(5, ['similarity scores']))

In [118]:
simple_search(0,df)

Leviton 5050 B01-0-000 Electrical Receptacle, 125/250 Vac, 50 A, 3 Pole, 3 Wire, Pack of 1, Black

      similarity scores                                                tag
0              1.000000  Leviton 5050 B01-0-000 Electrical Receptacle, ...
2681           0.997126  EcoAqua EFF-6013A Replacement for GE MWF Smart...
2343           0.997106  Deflecto Dryer Vent Hood, Wide Mouth, 4", Whit...
77             0.996828  Supco RIM500 Universal Modular Ice Maker, Repl...
3867           0.996659            Electrolux 218904301 O Ring Replacement


In [119]:
df["title"][77]

'Supco RIM500 Universal Modular Ice Maker, Replaces Whirlpool 4317943, 4211173, 4210317, 625601, 797991, 8114, W10122496'

In [120]:
#advanced search 
#first apply kmeans, get centroids
#search in the centroids

In [121]:
embeddings=df["final embeddings"].to_numpy()

In [122]:
for i in range(len(embeddings)):
    embeddings[i]=np.array(embeddings[i])

In [123]:
# list of numpy array into single numpy array
embeddings=np.stack(embeddings, axis=0)

In [124]:
embeddings.shape

(6040, 256)

In [ ]:
#ANN search performed

In [140]:
import annoy

In [163]:
class AnnoyIndex():
    def __init__(self, vectors, labels):
        print(vectors.shape)
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, number_of_trees=20):
        self.index = annoy.AnnoyIndex(self.dimension)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k, 
                                               search_k=5)                                           
        return [self.labels[i] for i in indices]

In [164]:
index = AnnoyIndex(embeddings,df["title"])
index.build()

(6040, 256)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [165]:
index.query(embeddings[0])

['Leviton 5050 B01-0-000 Electrical Receptacle, 125/250 Vac, 50 A, 3 Pole, 3 Wire, Pack of 1, Black',
 'EcoAqua EFF-6013A Replacement for GE MWF SmartWater, MWFA, MWFP, GWF, GWFA, Kenmore 469991 Refrigerator Water Filter',
 'Deflecto Dryer Vent Hood, Wide Mouth, 4", White (RVHAW4)',
 'Supco RIM500 Universal Modular Ice Maker, Replaces Whirlpool 4317943, 4211173, 4210317, 625601, 797991, 8114, W10122496',
 'Electrolux 218904301 O Ring Replacement',
 'Jenn-Air 6" Range Cooktop Stove Replacement Surface Burner Heating Element 12001231',
 'Samsung Refrigerator Assembly Guard Right DA97-11520b',
 'Whirlpool W10144820 Valve for Washer',
 'var aPageStart = (new Date()).getTime();\nvar ue_t0=ue_t0||+new Date();\n\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a])

In [169]:
dataf=pd.read_csv("final_df.csv")

In [170]:
dataf.shape

(25327, 19)